# Nuclear Localization Quantification

## Imports

In [ ]:
import numpy as np
import cv2
import pandas as pd
import matplotlib.pyplot as plt
from itertools import product
import os
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from mpl_toolkits import mplot3d
from scipy.optimize import curve_fit
import scipy.stats as st

## Process Images

In [ ]:
def findIntensities(images, nucleusColor, signalColor):
    # Define arrays to store distances from nuclei and signal intensities for all cells
    allDist = []
    allSignal = []
    allAverage = pd.DataFrame()
    
    # Define color dictionary and 
    colors = {'blue': 0, 'green': 1, 'red': 2, 'all':-1}
    maskColor = colors[nucleusColor]
    relevantColor = colors[signalColor]

    # Loop through all images
    for image in images:
        if image.endswith(".jpg"):
            image_path = path + '/' + image
            img = cv2.imread(image_path)
            
            # Determine gray image based on relevant channels
            if maskColor == -1:
                gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            else:
                gray = img[:,:,maskColor]
            
            # Apply blur, erosion, and thresholding
            blur = cv2.GaussianBlur(gray, (7,7), 0)
            kernel = np.ones((11,11), np.uint8)
            erode = cv2.erode(blur, kernel, iterations=1)
            ret, binImg = cv2.threshold(erode, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
            
            # Obtain contours, with largest one being that of the nucleus
            contours, hierarchy = cv2.findContours(binImg, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
            contours.sort(key=len, reverse=True)
            nucBound = contours[0]
            print(type(nucBound))
            print(nucBound)
            
            # Define arrays to store values for this nucleus
            oneNucDist = []
            oneNucSignal = []
            
            # Loop through all points. If in nulcear bound, find distance to contour and intensity at that point
            for y in range(gray.shape[0]):
                for x in range(gray.shape[1]):
                    dist = cv2.pointPolygonTest(nucBound, (x, y), True)
                    if dist >= 0:
                        oneNucDist.append(dist)
                        signal = img[y, x, relevantColor]
    #                     img[y,x,:] = [255, 255, 255]
                        oneNucSignal.append(signal)
    #         plt.figure(figsize=(3, 3))
    #         plt.imshow(img[:,:,::-1])
    
            # Normalize all distances to the maximum value
            maxDist = max(oneNucDist)
            oneNucDist = np.asarray(oneNucDist) / maxDist
            
            # Compute moving average calculation for each and add to data frame
            numberBins = 30
            averageSignal = binValues(oneNucDist, oneNucSignal, numberBins)
            allAverage[image] = averageSignal
            
            # Add this nucleus' data to that of all others
            allDist.append(oneNucDist)
            allSignal.append(oneNucSignal)
            
            # Visualize the boundary on the nucleus
#             cv2.drawContours(img, nucBound, -1, (0, 0, 255), 1)
#             plt.figure(figsize=(3, 3))
#             plt.imshow(img[:,:,::-1])
            
    # Compute the Average of Bins to determine the trend and confidence interval
    means = allAverage.mean(axis=1)
    lower_interval = []
    upper_interval = []
    for i in range(len(allAverage)):
        interval = st.t.interval(alpha=0.95, df=len(means[:])-1, loc=means[i], scale=st.sem(allAverage.iloc[i][:]))
        lower_interval.append(interval[0])
        upper_interval.append(interval[1])
    allAverage['mean'] = means
    allAverage['lower'] = lower_interval
    allAverage['upper'] = upper_interval
            
    return allAverage

In [ ]:
# Allow for customization of the number of bins - within each bin, average of all points
# Bin with key value is all the points with that key value up to the next key
def binValues(oneNucDist, oneNucSignal, numberBins):
    distValues = {}
    bins = np.linspace(0, numberBins - 1, numberBins)
    bins = np.round(bins, 0)
    for b in bins:
        distValues[b] = []
    for i in range(len(oneNucDist)):
        dist = np.floor(oneNucDist[i] * numberBins)
        signal = oneNucSignal[i]
        if dist == numberBins:
            distValues[numberBins-1].append(signal)
        else:
            distValues[dist].append(signal)
    keys = list(distValues.keys())
    keys.sort()
    averageSignal = [np.mean(distValues[key]) for key in keys]
    return averageSignal 

## Dancing Project Data

### Lamin A/C Day 1 Chonrogenesis

In [ ]:
# SG
path = ""
images = os.listdir(path)
images.sort()
df_Lamin_SG = findIntensities(images, 'all', 'green')

In [ ]:
# CG
path = ""
images = os.listdir(path)
images.sort()
df_Lamin_CG = findIntensities(images, 'all', 'green')

In [ ]:
df_Lamin_SG.to_csv("Nuclear Localization/Final Data/df_Lamin_SG.csv")
df_Lamin_CG.to_csv("Nuclear Localization/Final Data/df_Lamin_CG.csv")

In [ ]:
font = {'family' : 'normal',
        'weight' : 'bold',
        'size'   : 50}

mpl.rc('font', **font)

In [ ]:
# d1 = df_Lamin_SG
# d2 = df_Lamin_CG

d1 = pd.read_csv("Nuclear Localization/Final Data/df_Lamin_SG.csv")
d2 = pd.read_csv("Nuclear Localization/Final Data/df_Lamin_CG.csv")

plt.figure(figsize=(8, 8), dpi=300)
plt.style.use('default')

plt.plot(np.linspace(0,1,29), d1['mean'][:-1], color = '#0000FF', label="SG n=" + str(len(d1.columns)-3))
plt.fill_between(np.linspace(0,1,29), d1['lower'][:-1], d1['upper'][:-1], color='#0000FF', alpha=0.2)

plt.plot(np.linspace(0,1,29), d2['mean'][:-1], color='#FF0000', label="CG n=" + str(len(d2.columns)-3))
plt.fill_between(np.linspace(0,1,29), d2['lower'][:-1], d2['upper'][:-1], color='#FF0000', alpha=0.2)

plt.xlabel('Distance from the Nuclear Boundary', fontsize=20)
plt.ylabel('Signal Intensity', fontsize=20)
plt.title('Lamin A/C Singal Localization',fontsize=22)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.legend(loc="upper right", fontsize=20)
plt.xlim(0, 1)
plt.savefig("Nuclear Localization/Figures/Lamin Chondro D1.png", dpi = 300)
plt.show()

### H3K9me3 Day 1 Chonrogenesis

In [ ]:
# SG
path = ""
images = os.listdir(path)
images.sort()
df_H3K9me3_SG = findIntensities(images, 'blue', 'green')

In [ ]:
# CG
path = ""
images = os.listdir(path)
images.sort()
df_H3K9me3_CG = findIntensities(images, 'blue', 'green')

In [ ]:
df_H3K9me3_SG.to_csv("Nuclear Localization/Final Data/df_H3K9me3_SG.csv")
df_H3K9me3_CG.to_csv("Nuclear Localization/Final Data/df_H3K9me3_CG.csv")

In [ ]:
# d1 = df_H3K9me3_SG
# d2 = df_H3K9me3_CG

d1 = pd.read_csv("Nuclear Localization/Final Data/df_H3K9me3_SG.csv")
d2 = pd.read_csv("Nuclear Localization/Final Data/df_H3K9me3_CG.csv")

plt.figure(figsize=(8, 8), dpi=300)
plt.style.use('default')

plt.plot(np.linspace(0,1,29), d1['mean'][:-1], color = '#0000FF', label="SG n=" + str(len(d1.columns)-3))
plt.fill_between(np.linspace(0,1,29), d1['lower'][:-1], d1['upper'][:-1], color='#0000FF', alpha=0.2)

plt.plot(np.linspace(0,1,29), d2['mean'][:-1], color='#FF0000', label="CG n=" + str(len(d2.columns)-3))
plt.fill_between(np.linspace(0,1,29), d2['lower'][:-1], d2['upper'][:-1], color='#FF0000', alpha=0.2)

plt.xlabel('Distance from the Nuclear Boundary', fontsize=20)
plt.ylabel('Signal Intensity', fontsize=20)
plt.title('H3K9me3 Signal Localization', fontsize=22)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.legend(loc="upper right", fontsize=20)
plt.xlim(0, 1)
plt.savefig("Nuclear Localization/Figures/H3K9me3 Chondro D1.png", dpi = 300)
plt.show()

### H3K27me3 Day 1 Chonrogenesis

In [ ]:
# SG
path = ""
images = os.listdir(path)
images.sort()
df_H3K27me3_SG = findIntensities(images, 'blue', 'green')

In [ ]:
# CG
path = ""
images = os.listdir(path)
images.sort()
df_H3K27me3_CG = findIntensities(images, 'blue', 'green')

In [ ]:
df_H3K27me3_SG.to_csv("Nuclear Localization/Final Data/df_H3K27me3_SG.csv")
df_H3K27me3_CG.to_csv("Nuclear Localization/Final Data/df_H3K27me3_CG.csv")

In [ ]:
# d1 = df_H3K27me3_SG
# d2 = df_H3K27me3_CG

d1 = pd.read_csv("Nuclear Localization/Final Data/df_H3K27me3_SG.csv")
d2 = pd.read_csv("Nuclear Localization/Final Data/df_H3K27me3_CG.csv")

plt.figure(figsize=(8, 8), dpi=300)
plt.style.use('default')

plt.plot(np.linspace(0,1,29), d1['mean'][:-1], color = '#0000FF', label="SG n=" + str(len(d1.columns)-3))
plt.fill_between(np.linspace(0,1,29), d1['lower'][:-1], d1['upper'][:-1], color='#0000FF', alpha=0.2)

plt.plot(np.linspace(0,1,29), d2['mean'][:-1], color='#FF0000', label="CG n=" + str(len(d2.columns)-3))
plt.fill_between(np.linspace(0,1,29), d2['lower'][:-1], d2['upper'][:-1], color='#FF0000', alpha=0.2)

plt.xlabel('Distance from the Nuclear Boundary', fontsize=20)
plt.ylabel('Signal Intensity', fontsize=20)
plt.title('H3K27me3 Signal Localization', fontsize=22)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.legend(loc="upper right", fontsize=20)
plt.xlim(0, 1)
plt.savefig("Nuclear Localization/Figures/H3K27me3 Chondro D1.png", dpi = 300)
plt.show()

### DAPI Day 1 Chonrogenesis

In [ ]:
# SG
path = ""
images = os.listdir(path)
images.sort()
df_DAPI_SG = findIntensities(images, 'blue', 'blue')

In [ ]:
# CG
path = ""
images = os.listdir(path)
images.sort()
df_DAPI_CG = findIntensities(images, 'blue', 'blue')

In [ ]:
df_DAPI_SG.to_csv("Nuclear Localization/Final Data/df_DAPI_SG.csv")
df_DAPI_CG.to_csv("Nuclear Localization/Final Data/df_DAPI_CG.csv")

In [ ]:
d1 = df_DAPI_SG
d2 = df_DAPI_CG

plt.figure(figsize=(8, 8), dpi=300)
plt.style.use('default')

plt.plot(np.linspace(0,1,29), d1['mean'][:-1], color = '#0000FF', label="SG n=" + str(len(d1.columns)-3))
plt.fill_between(np.linspace(0,1,29), d1['lower'][:-1], d1['upper'][:-1], color='#0000FF', alpha=0.2)

plt.plot(np.linspace(0,1,29), d2['mean'][:-1], color='#FF0000', label="CG n=" + str(len(d2.columns)-3))
plt.fill_between(np.linspace(0,1,29), d2['lower'][:-1], d2['upper'][:-1], color='#FF0000', alpha=0.2)

plt.xlabel('Relative Distance from the Nuclear Boundary')
plt.ylabel('Signal Intensity')
plt.title('DAPI Singal Localization')
plt.legend(loc="upper right")
plt.xlim(0, 1)
plt.savefig("Nuclear Localization/Figures/DAPI Chondro D1.png", dpi = 300)
plt.show()

### H3K27ac Day 1 Chonrogenesis

In [ ]:
# SG
path = ""
images = os.listdir(path)
images.sort()
df_H3K27ac_SG = findIntensities(images, 'blue', 'green')

In [ ]:
# CG
path = ""
images = os.listdir(path)
images.sort()
df_H3K27ac_CG = findIntensities(images, 'blue', 'green')

In [ ]:
df_H3K27ac_SG.to_csv("Nuclear Localization/Final Data/df_H3K27ac_SG.csv")
df_H3K27ac_CG.to_csv("Nuclear Localization/Final Data/df_H3K27ac_CG.csv")

In [ ]:
d1 = df_H3K27ac_SG
d2 = df_H3K27ac_CG

plt.figure(figsize=(8, 8), dpi=300)
plt.style.use('default')

plt.plot(np.linspace(0,1,29), d1['mean'][:-1], color = '#0000FF', label="SG n=" + str(len(d1.columns)-3))
plt.fill_between(np.linspace(0,1,29), d1['lower'][:-1], d1['upper'][:-1], color='#0000FF', alpha=0.2)

plt.plot(np.linspace(0,1,29), d2['mean'][:-1], color='#FF0000', label="CG n=" + str(len(d2.columns)-3))
plt.fill_between(np.linspace(0,1,29), d2['lower'][:-1], d2['upper'][:-1], color='#FF0000', alpha=0.2)

plt.xlabel('Relative Distance from the Nuclear Boundary')
plt.ylabel('Signal Intensity')
plt.title('H3K27ac Singal Localization')
plt.legend(loc="upper right")
plt.xlim(0, 1)
plt.savefig("Nuclear Localization/Figures/H3K27ac Chondro D1.png", dpi = 300)
plt.show()

### AcK Day 1 Chonrogenesis

In [ ]:
# SG
path = ""
images = os.listdir(path)
images.sort()
df_AcK_SG = findIntensities(images, 'blue', 'green')

In [ ]:
# CG
path = ""
images = os.listdir(path)
images.sort()
df_AcK_CG = findIntensities(images, 'blue', 'green')

In [ ]:
df_AcK_SG.to_csv("Nuclear Localization/Final Data/df_AcK_SG.csv")
df_AcK_CG.to_csv("Nuclear Localization/Final Data/df_AcK_CG.csv")

In [ ]:
d1 = df_AcK_SG
d2 = df_AcK_CG

plt.figure(figsize=(8, 8), dpi=300)
plt.style.use('default')

plt.plot(np.linspace(0,1,29), d1['mean'][:-1], color = '#0000FF', label="SG n=" + str(len(d1.columns)-3))
plt.fill_between(np.linspace(0,1,29), d1['lower'][:-1], d1['upper'][:-1], color='#0000FF', alpha=0.2)

plt.plot(np.linspace(0,1,29), d2['mean'][:-1], color='#FF0000', label="CG n=" + str(len(d2.columns)-3))
plt.fill_between(np.linspace(0,1,29), d2['lower'][:-1], d2['upper'][:-1], color='#FF0000', alpha=0.2)

plt.xlabel('Relative Distance from the Nuclear Boundary')
plt.ylabel('Signal Intensity')
plt.title('AcK Singal Localization')
plt.legend(loc="upper right")
plt.xlim(0, 1)
plt.savefig("Nuclear Localization/Figures/AcK Chondro D1.png", dpi = 300)
plt.show()